In [ ]:
pip list | grep tensor

In [ ]:
import os
os.chdir('..')

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
# import torch
# import torchvision
# from pytorch_lightning.callbacks import ModelCheckpoint
# from src.model.lit_module import LitModule
# from src.data.dataset import VideoLabelDataset
import src.constants as const
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# from torch.utils.data import DataLoader
# from src.data.dataset import (VideoLabelDataset,
#                               VideoFolderPathToTensor,
#                               VideoResize)
import plotly
import numpy as np
import pandas as pd
import yaml
import os
# import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
%load_ext autoreload
%autoreload 2

In [ ]:
!which python

In [ ]:
def get_df_from_selection_bias_logs(version_str='lightning_logs/version_11/'):
    dirs = [d for d in os.listdir(version_str) if 'selection_bias' in d]
    event_file_paths = []
    for d in dirs:
        file = os.listdir(f'{version_str}{d}')[0]
        event_file_paths.append(f'{version_str}{d}/{file}')
    data = {}
    for p, d in zip(event_file_paths, dirs):
        val = []
        for event in tf.train.summary_iterator(p):
            for value in event.summary.value:
                if value.HasField('simple_value'):
                    val.append(value.simple_value)
        data.update({d:val})
    return pd.DataFrame(data)

def rename_columns(bias_df):
    return bias_df.rename(columns={
        "selection_bias_dec_0_lat_neu0": "selection_bias_dec0_lat_neu0",
        "selection_bias_dec_0_lat_neu2": "selection_bias_dec0_lat_neu2",
        "selection_bias_dec_0_lat_neu1": "selection_bias_dec0_lat_neu1"
    })

In [ ]:
df_bias = get_df_from_selection_bias_logs()
df_bias = rename_columns(df_bias)
df_bias.head()

In [ ]:
decoder_strs = [f'dec{c}' for c in range(len(const.HIDDEN_STATE_COLS))]
decoder_strs_dct = {d: sorted([c for c in df_bias.columns if d in c]) for d in decoder_strs}
decoder_strs_dct

In [ ]:
10000/3

In [ ]:
fig = make_subplots(rows=1, cols=3, shared_yaxes=True)
colors = ['#1f77b4','#ff7f0e', '#2ca02c']
for i, d in enumerate(decoder_strs):
    for c, color in zip(decoder_strs_dct[d], colors):
        fig.add_trace(go.Scatter(x=df_bias.index, y=df_bias[c],
                            mode='lines',
                            marker_color=color,
                            name=c.replace('_', ' ')), row=1, col=i+1)
fig.update_xaxes(title_text='Training step')
fig.update_layout(title_text="Selection bias during training", width=1800)
fig.update_yaxes(title_text="Selection bias activation", col=1)
fig.show()

In [ ]:
df_bias_ext = get_df_from_selection_bias_logs(version_str='lightning_logs/version_16/')
df_bias_ext = rename_columns(df_bias_ext)
fig = make_subplots(rows=1, cols=3, shared_yaxes=True)
colors = ['#1f77b4','#ff7f0e', '#2ca02c']
for i, d in enumerate(decoder_strs):
    for c, color in zip(decoder_strs_dct[d], colors):
        fig.add_trace(go.Scatter(x=df_bias_ext.index, y=df_bias_ext[c],
                            mode='lines',
                            marker_color=color,
                            name=c.replace('_', ' ')), row=1, col=i+1)
fig.update_xaxes(title_text='Training step')
fig.update_layout(title_text="Selection bias during training", width=1500)
fig.update_yaxes(title_text="Selection bias activation", col=1)
fig.show()

In [ ]:
df_new = df_bias.append(df_bias_ext).reset_index(drop=True)

In [ ]:
df_new.info()

In [ ]:
fig = make_subplots(rows=1, cols=3, shared_yaxes=True)
colors = ['#1f77b4','#ff7f0e', '#2ca02c']
for i, d in enumerate(decoder_strs):
    for c, color in zip(decoder_strs_dct[d], colors):
        fig.add_trace(go.Scatter(x=df_new.index, y=df_new[c],
                            mode='lines',
                            marker_color=color,
                            name=c.replace('_', ' ')), row=1, col=i+1)
fig.update_xaxes(title_text='Training step')
fig.update_layout(title_text="Selection bias during training", width=1500)
fig.update_yaxes(title_text="Selection bias activation", col=1)
fig.show()

In [ ]:
from graphviz import Digraph
import torch
from torch.autograd import Variable


def make_dot(var, params):
    """ Produces Graphviz representation of PyTorch autograd graph
    
    Blue nodes are the Variables that require grad, orange are Tensors
    saved for backward in torch.autograd.Function
    
    Args:
        var: output Variable
        params: dict of (name, Variable) to add names to node that
            require grad (TODO: make optional)
    """
    param_map = {id(v): k for k, v in params.items()}
    print(param_map)
    
    node_attr = dict(style='filled',
                     shape='box',
                     align='left',
                     fontsize='12',
                     ranksep='0.1',
                     height='0.2')
    dot = Digraph(node_attr=node_attr, graph_attr=dict(size="12,12"))
    seen = set()
    
    def size_to_str(size):
        return '('+(', ').join(['%d'% v for v in size])+')'

    def add_nodes(var):
        if var not in seen:
            if torch.is_tensor(var):
                dot.node(str(id(var)), size_to_str(var.size()), fillcolor='orange')
            elif hasattr(var, 'variable'):
                u = var.variable
                node_name = '%s\n %s' % (param_map.get(id(u)), size_to_str(u.size()))
                dot.node(str(id(var)), node_name, fillcolor='lightblue')
            else:
                dot.node(str(id(var)), str(type(var).__name__))
            seen.add(var)
            if hasattr(var, 'next_functions'):
                for u in var.next_functions:
                    if u[0] is not None:
                        dot.edge(str(id(u[0])), str(id(var)))
                        add_nodes(u[0])
            if hasattr(var, 'saved_tensors'):
                for t in var.saved_tensors:
                    dot.edge(str(id(t)), str(id(var)))
                    add_nodes(t)
    add_nodes(var.grad_fn)
    return dot

from torchvision import models
inputs = torch.randn(1,3,224,224)

resnet18 = models.resnet18()
y = resnet18(Variable(inputs))
# print(y)

g = make_dot(y, resnet18.state_dict())
g.view()

In [ ]:
videos